# Imports

In [1]:
from buspkg import DBFunctions as db
import pandas as pd

# Connect to DB

In [2]:
conn = db.GetDefaultConnection()
cur = conn.cursor()

## Removing Ghost Trips

Ghost trips are defined as trips that are impossible to have happened. The dataset has lots of records of the same bus at the same time, either to the same place or different. Since we are only interested in the actual trips that happened, we do not care about the reason why these ghost trips were registered in the database.

It was decided that only the last entry of a ghost trip would be kept in the database, even if it does not represent the trip that actually occured.

### Understanding ghost trips

In [3]:
query = """
    SELECT
        cnpj, plate, tripstartdatetime, COUNT(1) as c
    FROM 
        regulartrips
    GROUP BY
        tripstartdatetime, cnpj, plate, triptype
    HAVING
        COUNT(1) > 1
    ORDER BY
        c DESC
    LIMIT
        10;
"""

db.ExecuteQuery(query, cur, conn)
result = cur.fetchall()

As an example of ghost trips, we can see in the query result that the bus with plate `SHN3I44` performed **15 trips** starting at the same date and time. So, after removing ghost trips, only the record with the highest ID will be kept in the database.

In [4]:
result

[('23542573000142', 'SHN3I44', datetime.datetime(2024, 11, 3, 19, 0, 58), 15),
 ('76299270000107', 'AZZ8136', datetime.datetime(2023, 1, 14, 12, 22, 22), 10),
 ('25634551000138', 'PXV1649', datetime.datetime(2022, 9, 5, 1, 22, 31), 9),
 ('82647884000135', 'QIJ9054', datetime.datetime(2022, 12, 11, 8, 47, 51), 9),
 ('25634551000138', 'PKK8897', datetime.datetime(2022, 2, 20, 3, 24, 15), 9),
 ('23542573000142', 'PXC6881', datetime.datetime(2022, 8, 27, 14, 27, 50), 9),
 ('92667948000113', 'IYG5582', datetime.datetime(2022, 10, 6, 20, 14, 54), 9),
 ('25634551000138', 'PKQ6290', datetime.datetime(2022, 5, 22, 11, 16, 27), 9),
 ('30069314000101', 'KVW4H69', datetime.datetime(2022, 12, 10, 10, 14, 12), 9),
 ('76299270000107', 'BDR6E38', datetime.datetime(2023, 1, 13, 17, 50), 9)]

### Saving ghost trips data for later analysis

Although we don't want to have these records saved in our database, it is still interesting to know how many ghost trips each company had. This way we can confidently assess which company had the most true trips (unique).

In [5]:
query = """
    WITH
        GHOSTS AS (
            SELECT
                CNPJ, COUNT(1) AS C
            FROM
                REGULARTRIPS
            GROUP BY
                TRIPSTARTDATETIME, CNPJ, PLATE
        ),
        COMPANIES AS (
            SELECT
                CNPJ, SUM(C) - COUNT(1) AS GT, SUM(C) AS TT
            FROM
                GHOSTS
            GROUP BY
                CNPJ
        )
    SELECT
        COMPANIES.CNPJ, COMPANIES.GT, COMPANIES.TT
    FROM
        COMPANIES
"""

db.ExecuteQuery(query, cur, conn)
ghostTripsResult = cur.fetchall()

In [6]:
ghostTripsDF = pd.DataFrame(ghostTripsResult, columns=["CNPJ", "Ghost Trips", "Total Trips"])

ghostTripsDF["Ghost Trips"] = ghostTripsDF["Ghost Trips"].astype('int')
ghostTripsDF["Total Trips"] = ghostTripsDF["Total Trips"].astype('int')
ghostTripsDF["True Trips"] = ghostTripsDF["Total Trips"] - ghostTripsDF["Ghost Trips"]

ghostTripsDF = ghostTripsDF.drop(columns=["Total Trips"])
ghostTripsDF.describe()

,Ghost Trips,True Trips
count,213.000000,213.000000
mean,6531.572770,26047.253521
std,21252.720425,51527.901805
min,0.000000,11.000000
25%,4.000000,1405.000000
50%,49.000000,7522.000000
75%,1546.000000,25198.000000
max,156836.000000,348462.000000


In [7]:
ghostTripsList = list(ghostTripsDF.itertuples(index=False, name=None))
query = """
    UPDATE COMPANYINFO AS INFO
    SET
    	GHOSTTRIPS = DATA.GHOSTTRIPS,
    	TRUETRIPS = DATA.TRUETRIPS
    FROM
    	(VALUES %s) AS DATA(CNPJ, GHOSTTRIPS, TRUETRIPS)
    WHERE
    	INFO.CNPJ = DATA.CNPJ      
"""
db.ExecuteValues(query, cur, conn, ghostTripsList)
conn.commit()

## Removing Ghost Trips from regular trips table

Before any other data cleaning is done, we will remove all ghost trips from the regular trips table. This way, we can have more accurate data about the trips. 

Since this is a delicate operation, we will run a test to ensure that trips were removed correctly before commiting the updates.

In [8]:
query = """
    WITH
    	GHOSTS AS (
    		SELECT
    			T.CNPJ,
    			T.PLATE,
    			T.TRIPSTARTDATETIME,
    			MAX(T.ID) AS MAXID
    		FROM
    			REGULARTRIPS AS T
    		GROUP BY
    			T.CNPJ,
    			T.PLATE,
    			T.TRIPSTARTDATETIME
    		HAVING
    			COUNT(1) > 1
    	)
    DELETE FROM
        REGULARTRIPS AS T
    USING 
        GHOSTS AS G
    WHERE
        G.CNPJ = T.CNPJ
        AND G.PLATE = T.PLATE
        AND G.TRIPSTARTDATETIME = T.TRIPSTARTDATETIME
        AND T.ID < G.MAXID
"""
db.ExecuteQuery(query, cur, conn)

In [9]:
testQuery = """
    SELECT
        CNPJ, COUNT(1) AS C
    FROM
        REGULARTRIPS
    GROUP BY 
        CNPJ
"""
cur.execute(testQuery)
testResults = cur.fetchall()

testsDF = pd.DataFrame(testResults, columns=["CNPJ", "True Trips"])
testsDF = testsDF.set_index("CNPJ")

In [10]:
indexedGhostTrips = ghostTripsDF.set_index("CNPJ")
joinedDF = testsDF.join(indexedGhostTrips, how="inner", lsuffix=" Remaining", rsuffix=" Expected")

In [11]:
validation = joinedDF["True Trips Remaining"] == joinedDF["True Trips Expected"]
if validation.all():
    print("Ghost trips succesfully deleted.")
    conn.commit()
else:
    print("Error while deleting ghost trips.")
    conn.rollback()

Ghost trips succesfully deleted.


In [12]:
conn.close()